In [1]:
# VIX-Adjusted Momentum (US)

Replicating [VIX-Adjusted Momentum](https://cssanalytics.wordpress.com/2014/07/29/vix-adjusted-momentum/)

ERROR: Error in parse(text = x, srcfile = src): <text>:3:27: unexpected symbol
2: 
3: Replicating [VIX-Adjusted Momentum
                             ^


In [2]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registering fonts with R

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine


Attaching package

In [3]:
startDate <- as.Date('1990-01-02')
endDate <- as.Date('2019-06-30')

vixDt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^VIX' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()
    
sp500Dt <- indices$YahooFinanceTimeSeries() %>%
    filter(NAME == '^GSPC' & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

allXts <- merge(sp500Xts <- xts(sp500Dt$CLOSE, sp500Dt$TIME_STAMP), xts(vixDt$CLOSE, vixDt$TIME_STAMP))
names(allXts) <- c('SP500', 'VIX')


In [4]:
allXts$SMA_200 <- SMA(allXts$SP500, 200)
allXts$D_RET <- dailyReturn(allXts$SP500)
allXts$D_RET_LAG_1 <- stats::lag(allXts$D_RET, -1) #signal is generated and traded at the close, so use next day's return

allXts <- na.omit(allXts)

print(head(allXts))
print(tail(allXts))

            SP500   VIX  SMA_200         D_RET   D_RET_LAG_1
1990-10-15 303.23 31.64 338.9269  0.0106656401 -0.0142136261
1990-10-16 298.92 31.45 338.6230 -0.0142136261 -0.0005352703
1990-10-17 298.76 31.37 338.3230 -0.0005352703  0.0233631670
1990-10-18 305.74 29.60 338.0734  0.0233631670  0.0220449441
1990-10-19 312.48 27.86 337.8748  0.0220449441  0.0072964635
1990-10-22 314.76 27.27 337.6796  0.0072964635 -0.0076249362
             SP500   VIX  SMA_200        D_RET  D_RET_LAG_1
2019-06-20 2954.18 14.75 2775.676  0.009472185 -0.001259223
2019-06-21 2950.46 15.40 2775.945 -0.001259223 -0.001731887
2019-06-24 2945.35 15.26 2776.228 -0.001731887 -0.009496397
2019-06-25 2917.38 16.28 2776.425 -0.009496397 -0.001233934
2019-06-26 2913.78 16.21 2776.635 -0.001233934  0.003823176
2019-06-27 2924.92 15.82 2776.874  0.003823176  0.005757453


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)